# Day 09. Exercise 01
# Gridsearch

## 0. Imports

In [2]:
import pandas as pd
import itertools
from sklearn.metrics import accuracy_score 
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from tqdm.notebook import tqdm

## 1. Preprocessing

1. Read the file [`day-of-week-not-scaled.csv`](https://drive.google.com/file/d/1AlGvsJDSzPT_70caausx8bFuupIEZkfh/view?usp=sharing). It is similar to the one from the previous exercise, but this time we did not scale continuous features (we are not going to use logreg anymore). Don't forget to enrich the table with the 'dayofweek' column from the previous day's .csv-file.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [3]:
df = pd.read_csv('../../datasets/day-of-week-not-scaled.csv')
df_prev = pd.read_csv('../../datasets/dayofweek.csv')
df['dayofweek'] = df_prev['dayofweek'].values
df

,numTrials,hour,uid_user_0,uid_user_1,uid_user_10,uid_user_11,uid_user_12,uid_user_13,uid_user_14,uid_user_15,...,labname_lab03,labname_lab03s,labname_lab05s,labname_laba04,labname_laba04s,labname_laba05,labname_laba06,labname_laba06s,labname_project1,dayofweek
0,1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
1,2,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
2,3,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
3,4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
4,5,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681,9,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3
1682,6,20,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3
1683,7,20,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3
1684,8,20,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3


In [4]:
X=df.drop('dayofweek', axis=1)
y = df['dayofweek']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

## 2. SVM gridsearch

1. Using `GridSearchCV` try different parameters of kernel (`linear`, `rbf`, `sigmoid`), C (`0.01`, `0.1`, `1`, `1.5`, `5`, `10`), gamma (`scale`, `auto`), class_weight (`balanced`, `None`) use `random_state=21` and `probability=True` and get the best combination of them in terms of accuracy.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`. Check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [10]:
svc = SVC(probability=True, random_state=21)
param_grid = {'kernel': ['linear','rbf','sigmoid'],
             'C': [0.01, 0.1, 1, 1.5, 5, 10],
            'gamma': ['scale', 'auto'],
            'class_weight': ['balanced', None]}
grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, scoring='accuracy')
grid_search.fit(X_train, y_train)
grid_search.best_params_

{'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf'}

In [11]:
bestmod = grid_search.best_estimator_
y_pred = bestmod.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.8875739644970414

In [12]:
result = pd.DataFrame(grid_search.cv_results_)
result.sort_values(by='rank_test_score', inplace=True)
result = result[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']]
pd.set_option('display.max_colwidth', None)
result

,params,mean_test_score,std_test_score,rank_test_score
70,"{'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf'}",0.876109,0.018419,1
64,"{'C': 10, 'class_weight': 'balanced', 'gamma': 'auto', 'kernel': 'rbf'}",0.863500,0.010870,2
58,"{'C': 5, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf'}",0.816018,0.008116,3
52,"{'C': 5, 'class_weight': 'balanced', 'gamma': 'auto', 'kernel': 'rbf'}",0.807865,0.021257,4
63,"{'C': 10, 'class_weight': 'balanced', 'gamma': 'auto', 'kernel': 'linear'}",0.721052,0.034438,5
...,...,...,...,...
53,"{'C': 5, 'class_weight': 'balanced', 'gamma': 'auto', 'kernel': 'sigmoid'}",0.129792,0.019869,68
65,"{'C': 10, 'class_weight': 'balanced', 'gamma': 'auto', 'kernel': 'sigmoid'}",0.115693,0.020052,69
41,"{'C': 1.5, 'class_weight': 'balanced', 'gamma': 'auto', 'kernel': 'sigmoid'}",0.079380,0.006913,70
17,"{'C': 0.1, 'class_weight': 'balanced', 'gamma': 'auto', 'kernel': 'sigmoid'}",0.062310,0.002678,71


## 3. Decision tree

1. Using `GridSearchCV` try different parameters of `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use `random_state=21`.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [5]:
tree = DecisionTreeClassifier(random_state=21)
param_grid = {'max_depth': range(1,50),
            'criterion': ['entropy', 'gini'],
            'class_weight': ['balanced', None]}
grid_search_tree = GridSearchCV(estimator=tree, param_grid=param_grid, scoring='accuracy')
grid_search_tree.fit(X_train, y_train)
grid_search_tree.best_params_

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 23}

In [6]:
bestmod_tree = grid_search_tree.best_estimator_
y_pred_tree = bestmod_tree.predict(X_test)
accuracy_tree = accuracy_score(y_test, y_pred_tree)
accuracy_tree

0.893491124260355

In [7]:
result_tree = pd.DataFrame(grid_search_tree.cv_results_)
result_tree.sort_values(by='rank_test_score', inplace=True)
result_tree = result_tree[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']]
pd.set_option('display.max_colwidth', None)

## 4. Random forest

1. Using `GridSearchCV` try different parameters of `n_estimators` (`5`, `10`, `50`, `100`), `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use random_state=21.
2. Create a dataframe from the results of the gridsearch and sort it ascendengly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [8]:
RandForest = RandomForestClassifier(random_state=21)
param_grid = {'n_estimators': [5, 10, 50, 100],
            'max_depth':range(1,50),
            'class_weight': ['balanced', None],
            'criterion': ['entropy', 'gini'],
            }
grid_search_RandForest = GridSearchCV(estimator=RandForest, param_grid=param_grid, scoring='accuracy')
grid_search_RandForest.fit(X_train, y_train)
grid_search_RandForest.best_params_

{'class_weight': None,
 'criterion': 'gini',
 'max_depth': 28,
 'n_estimators': 50}

In [9]:
bestmod_forest = grid_search_RandForest.best_estimator_
y_pred_forest = bestmod_forest.predict(X_test)
accuracy_forest = accuracy_score(y_test, y_pred_forest)
accuracy_forest

0.9289940828402367

In [10]:
result_forest = pd.DataFrame(grid_search_RandForest.cv_results_)
result_forest.sort_values(by='rank_test_score', inplace=True)
result_forest = result_forest[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']]
pd.set_option('display.max_colwidth', None)

## 5. Progress bar

Gridsearch can be a quite long process and you may find yourself wondering when it will end.
1. Create a manual gridsearch for the same parameters values of random forest iterating through the list of the possible values and calculating `cross_val_score` for each combination. Try to increase `n_jobs`. The value `cv` for `cross_val_score` is 5.
2. Track the progress using the library `tqdm.notebook`.
3. Create a dataframe from the results of the gridsearch with the columns corresponding to the names of the parameters and `mean_accuracy` and `std_accuracy`.
4. Sort it descendingly by the `mean_accuracy`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [11]:
param_combinations = list(itertools.product(
    param_grid['n_estimators'],
    param_grid['max_depth'],
    param_grid['criterion'],
    param_grid['class_weight']
))

In [12]:
results = []
for n_estimators, max_depth, criterion, class_weight in tqdm(param_combinations, desc="Manual GridSearch"):
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, criterion=criterion, class_weight=class_weight, random_state=21, n_jobs=-1)
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy', n_jobs=-1)
    results.append({'n_estimators': n_estimators,
                  'max_depth': max_depth,
                  'criterion': criterion,
                  'class_weight': class_weight,
                  'mean_accuracy': scores.mean(),
                  'std_accuracy': scores.std()})

Manual GridSearch:   0%|          | 0/784 [00:00<?, ?it/s]

In [13]:
result_df = pd.DataFrame(results)
result_df.sort_values(by='mean_accuracy', ascending=False, inplace=True)
result_df.reset_index(drop=True, inplace=True)
result_df

,n_estimators,max_depth,criterion,class_weight,mean_accuracy,std_accuracy
0,50,28,gini,None,0.904290,0.010961
1,100,31,gini,None,0.904287,0.015204
2,50,45,gini,balanced,0.903549,0.012503
3,50,43,gini,balanced,0.903549,0.012503
4,50,44,gini,balanced,0.903549,0.012503
...,...,...,...,...,...,...
779,5,1,entropy,None,0.353832,0.016467
780,5,2,entropy,balanced,0.353110,0.021165
781,5,2,gini,balanced,0.346419,0.029749
782,5,1,gini,balanced,0.283390,0.011062


## 6. Predictions

1. Choose the best model and use it to make predictions for the test dataset.
2. Calculate the final accuracy.

In [14]:
RandForest = RandomForestClassifier(class_weight=None, criterion='gini',max_depth=28, n_estimators=50, random_state=21)
RandForest.fit(X_train, y_train)
pred = RandForest.predict(X_test)
accuracy_score(y_test, pred)

0.9289940828402367